# **class Client**

In [6]:
import torch
import threading
import queue
class Client:
  def __init__(self, client_id,partition1, partition, input_queue=None, data_loader=None):

        self.client_id = client_id
        self.partition = partition
        self.partition1 = partition1
        self.input_queue = input_queue
        self.data_loader = data_loader

  def process_partition1(self , data ):
    data_tensor = torch.tensor(data)
    return self.partition1(data_tensor)


  def process_partition(self , data ):
    data_tensor = torch.tensor(data)
    return self.partition(data_tensor)

  # calculate loss function
  def process_final_partition(self , data):
    print("calculate loss function here")


  def run(self):
    while True:
      if self.input_queue and not self.input_queue.empty():
        data = self.input_queue.get()
        x = data["output"]
        stage = data["stage"]
        source = data["source"]
        destination = data["destination"]
        client_id = data["client_id"]

        if self.client_id ==1 and ( stage !=1 and stage != 4):
          print("you shoulnt be here !!!")
          break

        # handle stage == 1
        if stage == 1 :
          output = self.process_partition1(x)
          stage +=1
          self.input_queue[stage].put({
               "output":output ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
          })
        elif stage !=5:
          output = self.process_partition(x)
          print(f"x value in client[{client_id}]stage:[{stage}]")
          stage +=1
          self.input_queue[stage].put({
                "output":output ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
            })
        else:
            print("final partition")
            self.process_final_partition(x)
      else:
        if self.data_loader:
          inputs , labels = self.data_loader.get_next_batch()
        self.input_queue[self.client_id].put({
            "output":inputs ,
            "stage":1,
            "source":self.client_id,
            "destination":self.client_id,
            "client_id":self.client_id
        })
      time.sleep(0.5)

# **Data Loader Class**

In [ ]:
import random
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader as TorchDataLoader

class DataLoader:
    def __init__(self, dataset, batch_size=32, shuffle=True, train_ratio=0.8):
        """
        Initialize the DataLoader with specific configurations.
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.train_ratio = train_ratio

        self.data_index = 0
        self.mode = "train"

        # set transform
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        self.dataset_train , self.dataset_test = self.loadDataSet()

        # create loader for test and train
        self.train_loader = TorchDataLoader(self.dataset_train , batch_size=self.batch_size , shuffle=self.shuffle)
        self.test_loader = TorchDataLoader(self.dataset_test , batch_size=self.batch_size , shuffle=False)
        self.mode = "train"

    def set_mode(self, mode):
        """
        train and test mod
        """
        if mode not in ["train", "test"]:
            raise ValueError("Mode must be either 'train' or 'test'")
        self.mode = mode

    def loadDataSet(self):
      trainset = torchvision.datasets.CIFAR10(root='./data',train=True ,download = True ,transform= self.transform)
      testset = torchvision.datasets.CIFAR10(root='./data', train=False ,download = True , transform=self.transform )
      return trainset , testset


    def getLoader(self):
      if self.mode == "train":
        return self.train_loader
      else:
        return self.test_loader

    def get_next_batch(self):
      loader = self.getLoader()
      return next(iter(loader))

# **define Partion class**

In [ ]:
class Partition:
    def __init__(self, in_channels, out_channels, num_blocks, stride=1):
        """
        Base class for a partition with configurable architecture.
        """
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_blocks = num_blocks
        self.stride = stride
        self.build_model()

In [ ]:
class Partition1(Partition):
    def build_model(self):
        # Define specific layers for Partition1
        pass

# **Message Class**

In [ ]:
class Message:
  def __init__(self , msg_type , stage, source , destination,data_id,data):
    self.msg_type = msg_type
    self.stage = stage
    self.source = source
    self.destination = destination
    self.data_id = data_id
    self.data = data

# **test DataLoader Class**

In [ ]:

data_loader = DataLoader(dataset='CIFAR10',batch_size=64, shuffle=True)


data_loader.set_mode("train")

inputs, labels = data_loader.get_next_batch()
print(f"Batch shape: {inputs.shape}, Labels shape: {labels.shape}")


data_loader.set_mode("test")
inputs, labels = data_loader.get_next_batch()
print(f"Batch shape: {inputs.shape}, Labels shape: {labels.shape}")


Files already downloaded and verified
Files already downloaded and verified
Batch shape: torch.Size([64, 3, 32, 32]), Labels shape: torch.Size([64])
Batch shape: torch.Size([64, 3, 32, 32]), Labels shape: torch.Size([64])


# **simulate logic Client Running with Thread**

In [2]:
import queue
import time
import threading

class Partition1:
    def process(self, client_id):
        print("partition 1")
        return client_id  #return client id

class Partition2:
    def process(self, x):
        print("partition 2")
        return x + 2

class Partition3:
    def process(self, x):
        print("partition 3")
        return x * 4

class Partition4:
    def process(self, x):
        print("partition 4")
        return x - 1

class FinalPartition:
    def process(self, x, label):
        print(f"Computed Value: {x}, Expected Label: {label}")
        if x == label:
            print("Success!")
        else:
            print("Failed!")



# Label‌ها
labels = {1: 11, 2: 15, 3: 19, 4: 23}

class Client(threading.Thread):
    def __init__(self, client_id, partition1, partition, final_partition  , input_queue ):
        threading.Thread.__init__(self)
        self.client_id = client_id
        self.partition1 = partition1
        self.partition = partition
        self.final_partition = final_partition
        self.input_queue = input_queue
    def run(self):
      while True:
          # check input list
        if not self.input_queue[self.client_id].empty():
          data = self.input_queue[self.client_id].get()
          x = data["output"]
          stage = data["stage"]
          source = data["source"]
          destination = data["destination"]
          client_id = data["client_id"]

          if self.client_id==1 and (stage !=1 and stage !=4):
            print(f'fuck................{stage}')
            break

          # data come from stage4
          # if stage == 4 and client_id==self.client_id:
          #   print(f"final partition for client ${self.client_id}")
          #   self.final_partition.process(x)
          if stage == 1:
            x = self.partition1.process(x)
            stage += 1
            self.input_queue[stage].put({
                "output":x ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
            })
          elif stage!=5:
            x = self.partition.process(x)
            print(f"x value in client[{client_id}]stage:[{stage}]={x}")
            stage +=1
            self.input_queue[stage].put({
                "output":x ,
                "stage":stage,
                "source":self.client_id,
                "destination":stage,
                "client_id":client_id
            })
          else:
            print("final partition")
            self.final_partition.process(x , labels[self.client_id])
        else:
          self.input_queue[self.client_id].put({
                "output":self.client_id ,
                "stage":1,
                "source":self.client_id,
                "destination":self.client_id,
                "client_id":self.client_id})

        time.sleep(5)
      # read from input




def print_queues(queues):
    for key, q in queues.items():
        items = list(q.queue)  # Accessing internal queue list (not thread-safe)
        print(f"Queue {key}: {items}")

In [3]:
import queue

queues = {
    1: queue.Queue(),
    2: queue.Queue(),
    3: queue.Queue(),
    4: queue.Queue(),
    5: queue.Queue()
}
partition1 = Partition1()
partition2 = Partition2()
partition3 = Partition3()
partition4 = Partition4()
final_partition = FinalPartition()


client1 = Client(1, partition1, None, final_partition, queues)
client2 = Client(2, partition1, partition2, final_partition, queues)
client3 = Client(3, partition1, partition3, final_partition, queues)
client4 = Client(4, partition1, partition4, final_partition, queues)

# runinng client parallel
clients = [client1, client2, client3, client4]
for client in clients:
    client.start()

for client in clients:
    client.join()


Exception in thread Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-10Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-d725768aaac4>", line 50, in run
    self.run()
  File "<ipython-input-2-d725768aaac4>", line 50, in run
    self.run()    self.run()
  File "<ipython-input-2-d725768aaac4>", line 50, in run

  File "<ipython-input-2-d725768aaac4>", line 50, in run
TypeError: 'Queue' object is not subscriptable
TypeError: 'Queue' object is not subscriptable
TypeError: 'Queue' object is not subscriptable
TypeError: 

In [ ]:


printQueue()

1
2
3
4


In [ ]:
import queue

# Sample data
queues = {
    1: queue.Queue(),
    2: queue.Queue(),
    3: queue.Queue(),
    4: queue.Queue()
}

# Add some items to the queues
queues[1].put("task1")
queues[2].put("task2")
queues[2].put("task3")
queues[3].put("task4")

# Print the contents of each queue


print_queues(queues)


Queue 1: ['task1']
Queue 2: ['task2', 'task3']
Queue 3: ['task4']
Queue 4: []
